In [2]:
pip install tqdm --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 732.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
#import all of the libraries 
#pip install tqdm
import torch
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import numpy as np
import copy
from tqdm import tqdm

#define Preceptron for mnist (same 28by28 pixels and 10 classes)
class SimplePerceptron(torch.nn.Module):
    def __init__(self):
        super(SimplePerceptron, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.linear = torch.nn.Linear(28*28, 10)  # 28x28 pixels to 10 classes

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        return x

#local model on client's dataset
#proximal term in the loss calculation to keep the local model close to the global model
#
def train_local_model(model, device, train_loader, optimizer, epoch, mu, global_weights):
    model.train()
    criterion = torch.nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        # Fixing the iteration over global_weights
        proximal_term = 0.0
        for param_key, param in model.named_parameters():
            # Ensure we're comparing the same parameters by their names
            global_param = global_weights[param_key]
            proximal_term += (mu / 2) * torch.norm(param - global_param) ** 2
        loss += proximal_term
        
        loss.backward()
        optimizer.step()

#evaluation of model
def test_model(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = torch.nn.CrossEntropyLoss(reduction='sum')
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)')
    return test_loss, accuracy

#fedprox-share training 
def fedprox_share_train(global_model, device, train_dataset, test_loader, epochs=1, mu=0.01, num_clients=10, frac=0.1):
    global_weights = global_model.state_dict() #initial weights
    client_indices = np.array_split(np.arange(len(train_dataset)), num_clients) 
    
    for epoch in range(epochs):
        local_weights = []
        m = max(int(frac * num_clients), 1)
        selected_clients = np.random.choice(range(num_clients), m, replace=False) #randomly selescts m clients
        
        for client in selected_clients:
            local_model = copy.deepcopy(global_model)
            local_model.to(device)
            optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01) #SGD
            
            train_loader = DataLoader(Subset(train_dataset, client_indices[client]), batch_size=64, shuffle=True)
            train_local_model(local_model, device, train_loader, optimizer, epoch, mu, global_weights) #train the local model on the client's data
            local_weights.append(local_model.state_dict())
        
        # Averaging local models
        global_weights = {key: torch.stack([local_weights[i][key] for i in range(len(local_weights))]).mean(0) for key in global_weights.keys()}
        global_model.load_state_dict(global_weights) #averaged weights are loaded back into the global model
        
        test_model(global_model, device, test_loader)

#main.py runs the model with given parameters
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # MNIST
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

    # Global model is preceptron
    global_model = SimplePerceptron().to(device)

    # fedprox-share training
    fedprox_share_train(global_model, device, train_dataset, test_loader, epochs=20, mu=0.01, num_clients=100, frac=0.1)


Test set: Average loss: 2.2216, Accuracy: 1905/10000 (19%)
Test set: Average loss: 2.1224, Accuracy: 3973/10000 (40%)
Test set: Average loss: 2.0326, Accuracy: 5499/10000 (55%)
Test set: Average loss: 1.9476, Accuracy: 6270/10000 (63%)
Test set: Average loss: 1.8717, Accuracy: 6631/10000 (66%)
Test set: Average loss: 1.7996, Accuracy: 6923/10000 (69%)
Test set: Average loss: 1.7338, Accuracy: 7126/10000 (71%)
Test set: Average loss: 1.6712, Accuracy: 7312/10000 (73%)
Test set: Average loss: 1.6137, Accuracy: 7407/10000 (74%)
Test set: Average loss: 1.5589, Accuracy: 7578/10000 (76%)
Test set: Average loss: 1.5093, Accuracy: 7637/10000 (76%)
Test set: Average loss: 1.4628, Accuracy: 7746/10000 (77%)
Test set: Average loss: 1.4195, Accuracy: 7809/10000 (78%)
Test set: Average loss: 1.3790, Accuracy: 7851/10000 (79%)
Test set: Average loss: 1.3426, Accuracy: 7905/10000 (79%)
Test set: Average loss: 1.3071, Accuracy: 7956/10000 (80%)
Test set: Average loss: 1.2746, Accuracy: 7985/10000 (80